In [ ]:
from qiskit import Aer, QuantumCircuit, transpile, assemble
from qiskit.circuit.library import RZGate, RZZGate
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import COBYLA
import numpy as np

# 定义问题参数
n = 2  # 操作数量
m = 2  # 机器数量
l = 2  # 二进制表示长度
K1 = 10.0  # 惩罚系数
P = 5.0  # 惩罚系数
p = np.array([[1, 2], [2, 1]])  # 处理时间矩阵

\begin{aligned}
&H=\frac{1}{2}\left(2^l-1\right)-\frac{1}{2} \sum_{h=0}^{l-1} 2^h Z_{z_h}\\&+K_1 \sum_{i=0}^{n-1}\left(\frac{(m-2)^2}{4}-\frac{(m-2)}{2} \sum_{j=0}^{m-1} Z_{x_{i j}}+\frac{1}{4}\left(\sum_{j=0}^{m-1} Z_{x_{i j}}\right)^2\right)\\&+P \sum_{j=0}^{m-1}\left(\frac{1}{4}\left(\sum_{i=0}^{n-1} p_{i j}-\left(2^l-1\right)\right)^2-\frac{1}{2}\left(\sum_{i=0}^{n-1} p_{i j}-\left(2^l-1\right)\right)\left(\sum_{i=0}^{n-1} p_{i j} Z_{x_{i j}}-\sum_{h=0}^{l-1} 2^h Z_{z_h}\right)\right)\left.+\frac{1}{4}\left(\sum_{i=0}^{n-1} p_{i j} Z_{x_{i j}}-\sum_{h=0}^{l-1} 2^h Z_{z_h}\right)^2\right)
\end{aligned}

In [ ]:
# 定义成本函数
def cost_function(params, n, m, l, K1, P, p):
    beta, gamma = params[:len(params)//2], params[len(params)//2:]
    num_qubits = n * m + l
    qc = QuantumCircuit(num_qubits, num_qubits)  # 添加经典寄存器

    # 初始化量子态到 |+> 态
    qc.h(range(n * m + l))

    # 应用 Cost Hamiltonian
    for g in gamma:
        # 单体项的 RZ 门
        for h in range(l):
            theta = g * 2 ** (h - 1)
            qc.rz(theta, n * m + h)

        for i in range(n):
            for j in range(m):
                theta = g * K1 * (m - 2) / 2
                qc.rz(theta, i * m + j)

        for j in range(m):
            theta = g * P / 2 * (np.sum(p[:, j]) - (2 ** l - 1))
            for i in range(n):
                qc.rz(theta, i * m + j)
        for j in range(m):
            theta = g * P / 2 * (np.sum(p[:, j]) - (2 ** l - 1))
            for h in range(l):
                qc.rz(theta, n * m + h)

        # 二体项的 RZZ 门
        for i in range(n):
            for j1 in range(m):
                for j2 in range(j1 + 1, m):
                    theta = g * K1 / 4
                    qc.rzz(theta, i * m + j1, i * m + j2)

        for j in range(m):
            for i1 in range(n):
                for i2 in range(i1 + 1, n):
                    theta = g * P / 4 * p[i1, j] * p[i2, j]
                    qc.rzz(theta, i1 * m + j, i2 * m + j)
        for j in range(m):
            for i in range(n):
                for h in range(l):
                    theta = g * P /4 *p[i, j] * 2 ** h
                    qc.rzz(theta, i * m + j, n * m + h)


        for h1 in range(l):
            for h2 in range(h1 + 1, l):
                theta = g * P / 4 * 2 ** h1 * 2 ** h2
                qc.rzz(theta, h1, h2)

    # 应用 XY 混合器
    # for b in beta:
    #     for i in range(n * m + l):
    #         for j in range(i + 1, n * m + l):
    #             qc.cx(i, j)
    #             qc.rx(2 * b, j)
    #             qc.cx(i, j)
    for b in beta:
        for i in range(n * m + l):
            qc.rx(2 * b, i)
    for i in range(n):
        for j in range(m-1):
            qc.rxx(np.pi/4, l + i * m + j, l + i * m + j + 1)
            qc.ryy(np.pi/4, l + i * m + j,l +  i * m + j + 1)


    # print(qc)
     # 模拟电路
     # 添加测量指令
    qc.measure(range(num_qubits), range(num_qubits))
    backend = Aer.get_backend('qasm_simulator')
    qc = transpile(qc, backend)
    qobj = assemble(qc, shots=10000)
    result = backend.run(qobj).result()
    counts = result.get_counts(qc)
    # 计算期望值
    expectation = 0
    for bitstring, count in counts.items():
        prob = count / 10000
        z_h = np.array([int(bit) for bit in bitstring[:l]])
        x_ij = np.array([int(bit) for bit in bitstring[l:]]).reshape(n, m)

        alpha_s = np.max([0, np.sum(x_ij * p) - np.sum([2 ** h * z for h, z in enumerate(z_h)])])
        E = np.sum([2 ** h * z for h, z in enumerate(z_h)]) + K1 * np.sum([(np.sum(x_ij[i]) - 1) ** 2 for i in range(n)]) + P * alpha_s ** 2
        expectation += E * prob

    return expectation, counts


\begin{aligned}
\tilde{E} =\sum_{h=0}^{l-1} 2^{h} z_{h}+K_{1} \sum_{i=1}^{n}\left(\sum_{j=1}^{m} x_{i j}-1\right)^{2}+ \sum_{j=1}^{m}  P \cdot \left( \max\{0, \sum_{i=1}^{n} x_{i j} p_{i j}-\sum_{h=0}^{l-1} 2^{h}z_{h}\} \right)^2
\end{aligned}

In [ ]:
# 优化 QAOA 参数
def optimize_qaoa(n, m, l, K1, P, p, params):
    def objective(params):
        expectation, _ = cost_function(params, n, m, l, K1, P, p)
        return expectation

    optimizer = COBYLA(maxiter=10000)
    # params = np.random.rand(2 * p_max) * 2 * np.pi
    result = optimizer.minimize(fun=objective, x0=params)

    return result

In [ ]:
import time

# 运行优化
# p_max = 6  # 最大层数

startTime = time.process_time()
ansatz_final = object()
init_point = np.array([])
counts = {}
min_energy = 0
final_result = object()
for p1 in range(1, 4):
    min_energy = 10000
    for k in range(50):
        print('第：', p1, '层', ' ', '第', k, '个参数')
        init_point_temp = init_point
        beta_p = np.random.uniform(0, np.pi)
        gamma_p = np.random.uniform(0,  2 * np.pi)
        if k >0:
            init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
            init_point_temp = np.delete(init_point_temp, p1 - 1)
        init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
        init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)
        # 创建参数向量
        result = optimize_qaoa(n, m, l, K1, P, p, init_point_temp)

        # print(result.x)
        optimal_params = result.x


        # 计算最终的能量
        energy, counts_temp = cost_function(result.x, n, m, l, K1, P, p)
        # 寻找这20对点中最优的参数
        if min_energy > energy:
            min_energy = energy
            init_point = init_point_temp
            counts = counts_temp
            final_result = result
endTime = time.process_time()
# params = np.random.rand(2 * p_max,2*np.pi)
# result = optimize_qaoa(n, m, l, K1, P, p, params)

In [ ]:
# 输出结果
print("Optimal parameters:", final_result.x)
print("Optimal value:", final_result.fun)
# 使用最优参数运行量子电路并输出测量结果
final_expectation, final_counts = cost_function(final_result.x, n, m, l, K1, P, p)
sorted_dict = dict(sorted(final_counts.items(), key=lambda item: item[1], reverse=True))
print("Final expectation value:", final_expectation)
print("Final measurement counts:", sorted_dict)